In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

events_df = pd.read_csv('../Data/GDELT_data/article_counts_disasters.csv')
prices_df = pd.read_csv('../Data/stock_prices_cleaned.csv')

In [2]:
events_df = events_df.drop([0,1])
events_df.tail()

,datetime,All Articles,MANMADE_DISASTER_IMPLIED,WB_2166_HEALTH_EMERGENCY_PREPAREDNESS_AND_DISASTER_RESPONSE,DISASTER_FIRE,NATURAL_DISASTER_ICE,NATURAL_DISASTER_EARTHQUAKE,NATURAL_DISASTER_FLOOD,NATURAL_DISASTER_FLOODING,NATURAL_DISASTER_HURRICANE,...,NATURAL_DISASTER_FRIGID,MANMADE_DISASTER_POWER_OUTAGE,NATURAL_DISASTER_HEATWAVE,NATURAL_DISASTER_AVALANCHES,NATURAL_DISASTER_TORRENTIAL_RAIN,SELF_IDENTIFIED_ENVIRON_DISASTER,NATURAL_DISASTER_ERUPTIONS,NATURAL_DISASTER_TWISTER,NATURAL_DISASTER_BUSHFIRE,NATURAL_DISASTER_FLASH_FLOOD
2132,2022-11-06 00:00:00+00:00,166754,30718,8280,2963,1681,603,1362,1310,626,...,30,84,61,28,145,75,31,43,8,3
2133,2022-11-07 00:00:00+00:00,281263,43567,14414,3791,2577,663,1659,2169,1373,...,31,115,61,42,62,65,22,68,30,32
2134,2022-11-08 00:00:00+00:00,306702,44237,15191,3234,2124,754,1771,2083,1924,...,40,80,42,53,75,78,31,85,74,51
2135,2022-11-09 00:00:00+00:00,327495,45854,18228,3363,2210,2120,1760,2171,2629,...,56,137,46,53,72,52,30,69,25,58
2136,2022-11-10 00:00:00+00:00,288809,43763,16023,3356,2209,933,1517,2373,3189,...,40,300,38,44,35,93,24,158,36,27


In [3]:
prices_df = prices_df.drop([0,1])
prices_df = prices_df.drop([len(prices_df)+1, len(prices_df), len(prices_df)-1])
prices_df.tail()

,Date,SHEL,CVX,BP,XOM,XEL,TSLA,IBDRY,JKS
2132,2022-11-03 00:00:00-04:00,55.234268,179.739639,32.687366,110.212761,65.150002,215.309998,39.340000,48.759998
2133,2022-11-04 00:00:00-04:00,56.552441,182.012054,33.557972,111.413094,65.550003,207.470001,39.939999,49.070000
2134,2022-11-05 00:00:00-04:00,56.552441,182.012054,33.557972,111.413094,65.550003,207.470001,39.939999,49.070000
2135,2022-11-06 00:00:00-04:00,56.552441,182.012054,33.557972,111.413094,65.550003,207.470001,39.939999,49.070000
2136,2022-11-07 00:00:00-05:00,56.354218,184.185242,33.963596,112.732475,65.250000,197.080002,39.880001,49.400002


In [4]:
prices_df.reset_index(drop = True, inplace=True)
events_df.reset_index(drop = True, inplace=True)
changes_df = prices_df.copy()

In [5]:
for column in prices_df:
    if(column == 'Date'):
        continue
    for i in range(1, len(prices_df)):
        change = (prices_df.loc[i-1, column] - prices_df.loc[i, column]) / prices_df.loc[i-1, column] 
        if(change > .005):
            changes_df.loc[i, column] = 1
        elif(change < -.01):
            changes_df.loc[i, column] = -1
        else:
            changes_df.loc[i, column] = 0
    changes_df.loc[0, column] = 0

In [6]:
changes_df = changes_df.drop('Date', axis=1)
events_df = events_df.drop('datetime', axis=1)

In [7]:
events_df.head()

,All Articles,MANMADE_DISASTER_IMPLIED,WB_2166_HEALTH_EMERGENCY_PREPAREDNESS_AND_DISASTER_RESPONSE,DISASTER_FIRE,NATURAL_DISASTER_ICE,NATURAL_DISASTER_EARTHQUAKE,NATURAL_DISASTER_FLOOD,NATURAL_DISASTER_FLOODING,NATURAL_DISASTER_HURRICANE,NATURAL_DISASTER_FLOODS,...,NATURAL_DISASTER_FRIGID,MANMADE_DISASTER_POWER_OUTAGE,NATURAL_DISASTER_HEATWAVE,NATURAL_DISASTER_AVALANCHES,NATURAL_DISASTER_TORRENTIAL_RAIN,SELF_IDENTIFIED_ENVIRON_DISASTER,NATURAL_DISASTER_ERUPTIONS,NATURAL_DISASTER_TWISTER,NATURAL_DISASTER_BUSHFIRE,NATURAL_DISASTER_FLASH_FLOOD
0,757916,129371,5765,14952,8754,3451,3366,3629,1222,1869,...,385,153,40,257,54,105,116,235,249,94
1,822862,134633,6477,12470,9553,5877,4216,2672,2723,3082,...,447,293,73,614,117,84,243,136,199,40
2,830028,143110,6919,11297,11397,3690,5559,4315,1679,4478,...,650,194,64,874,64,97,160,181,177,867
3,766969,136631,6445,9717,12375,3030,4043,2962,1500,2954,...,1074,448,69,1816,185,95,202,105,93,117
4,520691,102907,3600,8039,11862,1831,3206,2389,820,3029,...,640,126,47,520,125,57,84,61,49,53


In [8]:
from sklearn.model_selection import train_test_split

forest_models = []
forest_accuracies = []
for column in changes_df:  
    
    y = changes_df[column].to_numpy()
    
    X_train, X_test, y_train, y_test = train_test_split(events_df, y, 
                                                    test_size=0.2, 
                                                    random_state=0)
    
    model = RandomForestClassifier(n_jobs = -1)
    model.fit(X_train, y_train)
    forest_models.append(model)
    
    acc = model.score(X_test, y_test)
    forest_accuracies.append(acc)

In [9]:
from sklearn.neighbors import KNeighborsClassifier

knn_models = []
knn_accuracies = []
for column in changes_df:  
    
    y = changes_df[column].to_numpy()
    
    X_train, X_test, y_train, y_test = train_test_split(events_df, y, 
                                                    test_size=0.2, 
                                                    random_state=0)
    
    model = KNeighborsClassifier(n_neighbors=5, n_jobs = -1)
    model.fit(X_train, y_train)
    knn_models.append(model)
    
    acc = model.score(X_test, y_test)
    knn_accuracies.append(acc)

In [10]:
print(forest_accuracies)
print(knn_accuracies)

[0.6018735362997658, 0.6135831381733021, 0.5995316159250585, 0.5901639344262295, 0.6838407494145199, 0.4707259953161593, 0.6416861826697893, 0.4566744730679157]
[0.5761124121779859, 0.5550351288056206, 0.5714285714285714, 0.5644028103044496, 0.6252927400468384, 0.4379391100702576, 0.5737704918032787, 0.4566744730679157]


In [11]:
for column in events_df:
    if(column == 'All Articles'):
        continue
    else:
        events_df[column] = events_df[column] / events_df['All Articles']
events_df = events_df.drop('All Articles', axis=1)

In [12]:
ratio_forest_models = []
ratio_forest_accuracies = []
for column in changes_df:  
    y = changes_df[column].to_numpy()
    
    X_train, X_test, y_train, y_test = train_test_split(events_df, y, 
                                                    test_size=0.2, 
                                                    random_state=0)
    
    model = RandomForestClassifier(n_jobs = -1)
    model.fit(X_train, y_train)
    ratio_forest_models.append(model)
    
    acc = model.score(X_test, y_test)
    ratio_forest_accuracies.append(acc)

In [13]:
ratio_knn_models = []
ratio_knn_accuracies = []
for column in changes_df:  
    y = changes_df[column].to_numpy()
    
    X_train, X_test, y_train, y_test = train_test_split(events_df, y, 
                                                    test_size=0.2, 
                                                    random_state=0)
    
    model = KNeighborsClassifier(n_neighbors=5, n_jobs = -1)
    model.fit(X_train, y_train)
    ratio_knn_models.append(model)
    
    acc = model.score(X_test, y_test)
    ratio_knn_accuracies.append(acc)

In [14]:
print(ratio_forest_accuracies)
print(ratio_knn_accuracies)

[0.5831381733021077, 0.6112412177985949, 0.5901639344262295, 0.5807962529274004, 0.6838407494145199, 0.4613583138173302, 0.6206088992974239, 0.3981264637002342]
[0.5362997658079626, 0.5807962529274004, 0.5292740046838408, 0.5456674473067916, 0.6206088992974239, 0.3864168618266979, 0.5714285714285714, 0.3957845433255269]
